## 4. Predicción de fugas de clientes en una compañía telefónica (3.5 puntos)

Una compañía telefónica está interesada en que desarrollemos un modelo que prediga los **100 clientes actuales** (dataset de explotación) que tienen más probabilidad de abandonar la compañía. Para ello nos proporcionan una base de datos **fuga_clientes_empresa_telefonica_construccion.csv** (en carpeta data/) con casos etiquetados, que usaremos para construir nuestro modelo de predicción.

Los campos de esta base de datos son:

* **Customer ID**

* **network_age**: antigüedad del cliente en días

* **Customer tenure in months:** antigüedad del cliente en meses

* **Total Spend in Months 1 and 2:** gasto total del cliente en los meses de referencia 1 y 2

* **Total SMS Spend:** gasto total en SMS

* **Total Data Spend:** gasto total en datos/internet

* **Total Data Consumption:** consumo total de datos (en KB) durante el período de estudio

* **Total Unique Calls:** número total de llamadas únicas

* **Total Onnet spend:** gasto total en llamadas a otros usuarios de la misma red de telefonía

* **Total Offnet spend:** gasto total en llamadas a otros usuarios de redes de telefonía diferentes

* **Total Call centre complaint calls:** número de llamadas de queja al call center

* **Network type subscription in Month 1:** suscripción de tipo de red en el mes 1. Esto indica la suscripción de red preferida de un cliente, lo que puede indicar su tipo de dispositivo: servicio 2G o 3G

* **Network type subscription in Month 2:** igual que el anterior pero en el mes posterior

* **Churn Status**: el valor es 1 si el cliente abandona la compañía telefónica, 0 si permanece en ella

* **Most Loved Competitor network in Month 1:** qué otro proveedor de la competencia prefiere el cliente en el mes 1. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

* **Most Loved Competitor network in Month 2:** qué otro proveedor de la competencia prefiere el cliente en el mes 2. En realidad es un conjunto de columnas, cada una enfocada en un proveedor particular

La variable a predecir es **Churn Status**: el valor es 1 si el cliente **abandona** la compañía, 0 si no la abandona.

La compañía también nos proporciona otra base de datos, **fuga_clientes_empresa_telefonica_explotacion.csv**, con información sobre clientes de los que no se sabe ahora mismo si van a permanecer o no en la compañía. Por tanto en esta segunda base de datos todos los valores de la columna **Churn Status** son missing values (NaN).

La compañía nos pide que proporcionemos los IDs de los 100 clientes de la base de datos de explotación que con mayor probabilidad vayan a abandonar la compañía. Para ello proporcionaremos como entregable a la compañía un archivo csv con una sola columna, **Customer ID**, y 100 filas que contengan los IDs de los clientes seleccionados.

El fichero **ejemplo_fichero_predicciones.csv** contiene un ejemplo con el formato solicitado para este archivo.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


In [ ]:
datos_construccion = pd.read_csv("./data/fuga_clientes_empresa_telefonica_construccion.csv")
datos_explotacion  = pd.read_csv("./data/fuga_clientes_empresa_telefonica_explotacion.csv")

In [ ]:
datos_construccion.columns

In [ ]:
datos_explotacion.columns

In [ ]:
# chequeo de que las columnas son exactamente las mismas en los dos ficheros:
sum(datos_construccion.columns != datos_explotacion.columns)

In [ ]:
# la columna a predecir es "Churn Status"
# si es 1, el cliente se va de la compañía
# si es 0, el cliente se queda

# Esta columna se sabe en el dataset de construcción (ejemplos de clientes pasados):
datos_construccion["Churn Status"].values[:10]

In [ ]:
# Sin embargo no se sabe en el dataset de explotación (clientes actuales):
datos_explotacion["Churn Status"].values[:10]

In [ ]:
# División de datos en características y objetivo
x = datos_construccion.drop(['Customer ID', 'Churn Status'], axis=1)
y = datos_construccion['Churn Status']

In [ ]:
# Neural network
neural_network_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, alpha=0)
# K-nn algorithm
knn_clf = KNeighborsClassifier(n_neighbors=24)
# Gaussian naive bayes
naive_bayes_clf = GaussianNB()
# Logistic regression
logistic_reg_clf = LogisticRegression(C=100,solver='lbfgs', max_iter=1000)
# Decision tree
decision_tree_clf = DecisionTreeClassifier(max_depth=7, min_samples_leaf=4,min_samples_split=10)

# Testing neural network

scores = cross_val_score(neural_network_clf,x,y,cv=5,)

print("Score global del modelo neural network: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

# Testing k-nn algorithm

scores = cross_val_score(knn_clf,x,y,cv=5,)

print("Score global del modelo k-nn: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

# Testing gaussian naive bayes

scores = cross_val_score(naive_bayes_clf,x,y,cv=5,)


print("Score global del modelo gaussian: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

# Testing logistic regression

scores = cross_val_score(logistic_reg_clf,x,y,cv=5,)


print("Score global del modelo logistic regression: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

# Testing decision tree
scores = cross_val_score(decision_tree_clf,x,y,cv=5,)


print("Score global del modelo decission tree: {:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

In [ ]:
# Usamos decision tree, el modelo con mejor puntuación 
decision_tree_clf.fit(x, y)

# Predicción de probabilidades en el dataset de explotación
x_explotacion = datos_explotacion.drop(['Customer ID', 'Churn Status'], axis=1)
probabilidades_fuga = decision_tree_clf.predict_proba(x_explotacion)[:, 1]

# Seleccionar los 100 clientes con mayor probabilidad de fuga
ids_clientes = datos_explotacion['Customer ID']
indices_top_100 = np.argsort(probabilidades_fuga)[-100:]
top_100_clientes = ids_clientes.iloc[indices_top_100]

# Generar archivo CSV
top_100_clientes.to_csv("top_100_clientes_fuga.csv", index=False)